In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import yaml

data_yaml_content = {
    'train': '/content/drive/MyDrive/vegetation segmentation.v2i.yolov8/train',
    'test': '/content/drive/MyDrive/vegetation segmentation.v2i.yolov8/test',
    'val': '/content/drive/MyDrive/vegetation segmentation.v2i.yolov8/valid',
    'nc': 1, # number of classes
    'names': ['vegetation'] # class names
}

with open('data.yaml', 'w') as file:
    yaml.dump(data_yaml_content, file, default_flow_style=None)

print("data.yaml file created successfully.")

data.yaml file created successfully.


In [ ]:
!pip install ultralytics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.4 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Build YOLOv8 segmentation model from scratch
model = YOLO('yolov8n-seg.yaml')  # for segmentation
# model = YOLO('yolov8n.yaml')    # for detection (if you prefer detection)

# Train from scratch with early stopping
model.train(
    data="/content/data.yaml",
    epochs=200,          # maximum epochs
    imgsz=640,           # image size
    batch=32,            # batch size
    pretrained=False,    # start from scratch
    optimizer="AdamW",
    lr0=0.001,                   # learning rate
    augment=True,                # enable augmentations
    weight_decay=0.0005,         # L2 regularization
    cos_lr=True,
    patience=20,# early stopping (stop if no improvement in 10 epochs)
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c295b6e74a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

In [ ]:
metrics = model.val(data="/content/data.yaml", split="test")
print(metrics)


Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 11.3 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 0.4±0.2 MB/s, size: 141.1 KB)
val: Scanning /content/drive/MyDrive/vegetation segmentation.v2i.yolov8/test/labels.cache... 27 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 27/27 27.5Kit/s 0.0s
WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.1it/s 16.6s
                   all         27         80      0.558       0.55      0.586      0.345      0.514      0.537      0.525      0.273
Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/runs/segment/train2
ultralytics.utils.metrics.SegmentMetrics object with attribute

In [ ]:
# Extract precision and recall
precision = metrics.results_dict['metrics/precision(B)']
recall = metrics.results_dict['metrics/recall(B)']

# Compute F1 score
f1_score = 2 * (precision * recall) / (precision + recall)

# Extract mAP50 (accuracy proxy for object detection/segmentation)
map50 = metrics.results_dict['metrics/mAP50(B)']

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"mAP50 (Accuracy proxy): {map50:.4f}")

Precision: 0.5576
Recall: 0.5500
F1 Score: 0.5538
mAP50 (Accuracy proxy): 0.5865


In [ ]:
metrics = model.val(data="/content/data.yaml", split="val")

Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 46.2±21.0 MB/s, size: 166.2 KB)
val: Scanning /content/drive/MyDrive/vegetation segmentation.v2i.yolov8/valid/labels.cache... 59 images, 9 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 59/59 99.9Kit/s 0.0s
WARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 50% ━━━━━━────── 1/2 0.1it/s 3.1s<10.4sWARNING ⚠️ Model does not support 'augment=True', reverting to single-scale prediction.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.4it/s 5.0s
                   all         59        132      0.675      0.551      0.618      0.386      0.695      0.545       0.59      0.337
Speed: 6.7ms p

In [ ]:
# Extract precision and recall
precision = metrics.results_dict['metrics/precision(B)']
recall = metrics.results_dict['metrics/recall(B)']

# Compute F1 score
f1_score = 2 * (precision * recall) / (precision + recall)

# Extract mAP50 (accuracy proxy for object detection/segmentation)
map50 = metrics.results_dict['metrics/mAP50(B)']

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")
print(f"mAP50 (Accuracy proxy): {map50:.4f}")


Precision: 0.6751
Recall: 0.5510
F1 Score: 0.6068
mAP50 (Accuracy proxy): 0.6176
